In [ ]:
!pip install -q transformers sentencepiece gradio

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
import gradio as gr

model_name = "nilayshenai/BART-English-to-Bhojpuri-Alpha1"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def translate(text):
    tokenizer.src_lang = "en_XX"
    tgt_lang_code = tokenizer.lang_code_to_id["hi_IN"]
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        **inputs,
        forced_bos_token_id=tgt_lang_code,
        max_length=128,
        num_beams=5
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

custom_css = """
body {
    font-family: 'Inter', sans-serif;
}
h1, h2, h3, h4 {
    font-family: 'Inter', sans-serif;
    font-weight: 600;
}
textarea, input {
    font-family: 'Inter', sans-serif;
    font-size: 16px;
}
button {
    font-family: 'Inter', sans-serif;
    background-color: #1A73E8;
    color: white;
    border: none;
    padding: 12px 20px;
    border-radius: 8px;
    font-size: 16px;
    font-weight: 600;
    cursor: pointer;
}
button:hover {
    background-color: #1558b0;
}
"""
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown(
        """
        # Fine-tuned mBART for English-to-Bhojpuri Translation
        #### A research-driven attempt utilizing mBART fine-tuning for English-to-Bhojpuri translation, with a focus on low-resource language adaptation. The model has been trained on a custom-curated dataset developed specifically for this task.
        """
    )

    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(
                lines=6, 
                placeholder="Enter English text here...",
                label="English Text"
            )
            translate_btn = gr.Button("Translate")
        with gr.Column():
            output_text = gr.Textbox(
                lines=6, 
                label="Bhojpuri Translation", 
                interactive=False
            )

    translate_btn.click(fn=translate, inputs=input_text, outputs=output_text)

    gr.Markdown("<hr>")

    gr.Markdown(
        """<center><font size=2 color=gray>Nilay D. Shenai • Model Version Alpha 2</font></center>"""
    )

demo.launch(share=True, show_api=False)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://546f4ed60466fa702f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
